In [1]:
from keras.models import load_model, Model
from keras import backend as K
import pandas as pd
import numpy as np
from keras.layers import Input
import tensorflow as tf

Using TensorFlow backend.


In [2]:
test_df = pd.read_csv('../dataset/mnist-in-csv/mnist_test.csv')
m_test = len(test_df)
X_test = np.array(test_df.loc[:, test_df.columns != 'label'].values.reshape(m_test, 28, 28, 1), dtype=np.float64)
y_test = test_df['label'].values
X_test /= 255

In [3]:
model = load_model('trained-weights-model/MNIST-triplet-loss-network-andrew-ng.h5', compile=False, custom_objects={'tf': tf})

In [4]:
input_layer = Input((28,28,1))
extracted_network = model.get_layer(index=3)
extracted_network = extracted_network(input_layer)

In [5]:
model = Model(input_layer, extracted_network)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 1, 128)            45164     
Total params: 45,164
Trainable params: 45,164
Non-trainable params: 0
_________________________________________________________________


In [6]:
for i, j in enumerate(y_test[:40]):
    if i%10 == 0:
        print()
    print(str(i) + ' -> ' + str(j), end='  ')


0 -> 7  1 -> 2  2 -> 1  3 -> 0  4 -> 4  5 -> 1  6 -> 4  7 -> 9  8 -> 5  9 -> 9  
10 -> 0  11 -> 6  12 -> 9  13 -> 0  14 -> 1  15 -> 5  16 -> 9  17 -> 7  18 -> 3  19 -> 4  
20 -> 9  21 -> 6  22 -> 6  23 -> 5  24 -> 4  25 -> 0  26 -> 7  27 -> 4  28 -> 0  29 -> 1  
30 -> 3  31 -> 1  32 -> 3  33 -> 4  34 -> 7  35 -> 2  36 -> 7  37 -> 1  38 -> 2  39 -> 1  

In [7]:
image_a = X_test[39]
image_a = np.expand_dims(image_a, axis=0)
image_a_encoding = model.predict(image_a)
print(image_a_encoding.shape)

image_b = X_test[29]
image_b = np.expand_dims(image_b, axis=0)
image_b_encoding = model.predict(image_b)
print(image_b_encoding.shape)

# print(image_a_encoding)
# print(image_b_encoding)

dist = np.linalg.norm(image_a_encoding - image_b_encoding)
print(dist)

(1, 1, 128)
(1, 1, 128)
0.834716


In [8]:
model.save_weights('trained-weights-model/triplet_loss_encoding_network_weights.h5')
model.save('trained-weights-model/triplet_loss_encoding_network_model.h5')